In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [4]:
import numpy as np
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import best_rq

In [5]:
from malaya_speech.train.model.conformer.model import Model as ConformerModel

class Encoder:
    def __init__(self, config):
        self.config = config
        self.encoder = ConformerModel(**self.config)

    def __call__(self, x, input_mask, training = True):
        return self.encoder(x)
    
config_conformer = malaya_speech.config.conformer_base_encoder_config
config_conformer['subsampling']['type'] = 'none'
config_conformer['dropout'] = 0.0
encoder = Encoder(config_conformer)

In [6]:
# X = tf.placeholder(tf.float32, (None, None))
# X_len = tf.placeholder(tf.int32, (None,))

In [7]:
X = tf.placeholder(tf.float32, (None, None))
X_len = tf.placeholder(tf.int32, (None,))

In [8]:
cfg = best_rq.Best_RQConfig()
model = best_rq.Model(cfg, encoder)

In [ ]:
out = model(X, padding_mask = X_len)
out

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

In [ ]:
%%time

o = sess.run(out, feed_dict = {X: [y, y], X_len: [len(y), len(y)]})